# Classification of tetrahedra with rational dihedral angles

This notebook gives a computational classification of configurations of four vectors $\mathbf{v}_1, \mathbf{v}_2, \mathbf{v}_3, \mathbf{v}_4 \in \mathbb{R}^3$, no three coplanar, for which for every $j,k$, the angle $\theta_{jk}$ between $\mathbf{v}_j$ and $\mathbf{v}_k$ is a rational multiple of $\pi$. 
This includes the classification of Euclidean tetrahedra whose dihedral angles are rational multiples of $\pi$.

Taking $x_{jk} = e^{i \theta_{jk}}$, this corresponds to finding
solutions of the *Gram determinant equation*
$$
\det(M) = 0, \qquad M = \begin{pmatrix} 2 & x_{12} + x_{12}^{-1} & x_{13} + x_{13}^{-1} & x_{14} + x_{14}^{-1} \\
x_{12} + x_{12}^{-1} & 2 & x_{23} + x_{23}^{-1} & x_{24} + x_{24}^{-1} \\
x_{13} + x_{13}^{-1} & x_{23} + x_{23}^{-1} & 2 & x_{34} + x_{34}^{-1} \\
x_{14} + x_{14}^{-1} & x_{24} + x_{24}^{-1} & x_{34} + x_{34}^{-1} & 2
\end{pmatrix}
$$
in roots of unity, in which none of the variables equals $\pm 1$ and none of the $3 \times 3$ minors of $M$ vanishes; such solutions are said to be *nondegenerate*.

Since the quantity $\det(M)$ belongs to the Laurent polynomial ring
$$
\mathbb{Q}[x_{jk}^{\pm 2}, x_{jk} x_{jl} x_{kl}],
$$
we conflate solutions that correspond to the same prime ideal of this ring. The ideal $\det(M)$ is also invariant by the group of automorphisms of this ring generated by:
- the action of $S_4$ on $\{1,2,3,4\}$;
- the action of $(\mathbb{Z}/2\mathbb{Z})^6$ given by $x_{jk} \mapsto x_{jk}^{-1}$;
- the *Regge symmetry* 
$$
x_{13} \mapsto s x_{13}^{-1}, x_{14} \mapsto s x_{14}^{-1}, x_{23} \mapsto s x_{23}^{-1}, x_{24} \mapsto s x_{24}^{-1}  \qquad (s = (x_{13} x_{14} x_{23} x_{24})^{1/2}).
$$
This group maps to $W(D_6)$ (viewed as the group of signed $6 \times 6$ permutation matrices with an even number of minus signs) via its action on the monomials
$$
x_{12}^2 x_{34}^2,\,x_{12}^2 x_{34}^{-2},\, x_{13}^2 x_{24}^2, \,x_{13}^2 x_{24}^{-2}, \,x_{14}^2 x_{23}^2, \,x_{14}^2 x_{23}^{-2}.
$$
We thus further consider solutions to be equivalent if their corresponding prime ideals are $W(D_6)$-equivalent. We find that:
- there is a single equivalence class of one-parameter solutions;
- every isolated solution is either an isolated solution of the equation $\det(M) \equiv 0 \pmod{2}$ or is equivalent to a solution with values in $\mathbb{Q}(\zeta_N)$ where $N \in \{42, 48, 120\}$. (Note that the solution itself is only then guaranteed to have values in $\mathbb{Q}(\zeta_{2N})$. Note also that in the paper, denominators are measured using multiples of $\pi$ rather than $2\pi$.)

The mod-2 equation reads
$$
2 \cos (2\theta_{12} + 2 \theta_{34}) +
2 \cos (2\theta_{12} - 2 \theta_{34}) +
2 \cos (2\theta_{13} + 2 \theta_{24}) +
2 \cos (2\theta_{13} + 2 \theta_{24}) +
2 \cos (2\theta_{14} + 2 \theta_{23}) +
2 \cos (2\theta_{14} + 2 \theta_{23})
\equiv 0 \pmod{2}.
$$
We first identify solutions of this equation using the mod-2 analogue of the Poonen--Rubinstein classification of 6-term cosine relations. For each of the possible forms of such a relation, we obtain a collection of problems of the following form: compute the torsion closure of the ideal generated by $\det(M)$ plus some additional generators. (In some cases, we reduce this collection using the symmetry group identified above.) We then call the Sage function `torsion_closure` to compute the components of the torsion closure up to Galois symmetries; eliminate degenerate solutions; and then analyze the results.

This notebook was originally run using SageMath version 9.2. Allow roughly 2 hours on a single CPU to complete all computations.

There is a dependency on one external Sage file:
- `torsion_closure.sage`: implements a computation of torsion closures of ideals in Laurent polynomial rings, based on ideas of Ruppert, Beukers-Smyth, and Aliev-Smyth (plus the mod-2 analogue of the classification of short cyclotomic relatiosn from Poonen-Rubinstein).

## Preliminary definitions and declarations

Import some utility functions from the Python `itertools` module.

In [1]:
from itertools import chain, permutations 

Import the code to compute torsion closures of Laurent polynomial ideals.

In [2]:
load("torsion_closure.sage")

Construct the polynomial equation constraining dihedral angles of tetrahedra, which we refer to below as the *Gram equation*.

In [3]:
P.<x_12,x_34,x_13,x_24,x_14,x_23> = LaurentPolynomialRing(QQ, 6, order='degrevlex')

In [4]:
    # Note: in Sage, ~x is a shorthand for x^-1.
gram_matrix = Matrix([[2, x_12+~x_12, x_13+~x_13, x_14+~x_14],
                       [x_12+~x_12, 2, x_23+~x_23, x_24+~x_24],
                       [x_13+~x_13, x_23+~x_23, 2, x_34+~x_34],
                       [x_14+~x_14, x_24+~x_24, x_34+~x_34, 2]])
print(gram_matrix)

[             2 x_12 + x_12^-1 x_13 + x_13^-1 x_14 + x_14^-1]
[x_12 + x_12^-1              2 x_23 + x_23^-1 x_24 + x_24^-1]
[x_13 + x_13^-1 x_23 + x_23^-1              2 x_34 + x_34^-1]
[x_14 + x_14^-1 x_24 + x_24^-1 x_34 + x_34^-1              2]


In [5]:
gram_determinant = gram_matrix.det()
gram_ideal = P.ideal([gram_determinant])

Extract monomials not divisible by 2, then group them into conjugate pairs. These will be the pairs on which the action of $W(D_6)$ is most apparent.

In [6]:
ex = [e for e,c in gram_determinant.dict().items() if c%2]
monomials1 = [P.monomial(*e) for e in ex if e[[i for i in range(6) if e[i]][0]] > 0]
monomials = [(x, ~x) for x in monomials1]
monomials

[(x_12^2*x_34^2, x_12^-2*x_34^-2),
 (x_13^2*x_24^2, x_13^-2*x_24^-2),
 (x_14^2*x_23^2, x_14^-2*x_23^-2),
 (x_14^2*x_23^-2, x_14^-2*x_23^2),
 (x_13^2*x_24^-2, x_13^-2*x_24^2),
 (x_12^2*x_34^-2, x_12^-2*x_34^2)]

Compute ideals cutting out the components of the degenerate locus, and define a function to test whether an ideal is supported in the degenerate locus.

In [7]:
minors_ideals = []
for i in range(6):
    minors_ideals.append(P.ideal([P.gens()[i] - 1]))
    minors_ideals.append(P.ideal([P.gens()[i] + 1]))
for f in gram_matrix.minors(3):
    minors_ideals.append(P.ideal([f]))

def is_degenerate(J):
    ring = J.ring()
    return any(all(ring(f) in J for f in I.gens()) for I in minors_ideals)

Lazy shortcut: define aliases `x1, ..., x6` for the Laurent polynomial variables.

In [8]:
(x1, x2, x3, x4, x5, x6) = P.gens()

Construct geometric symmetries of the Gram equation.

In [9]:
geometric_symmetries = (
    (~x1, x2, x3, x4, x5, x6), # angle flip
    (x1, x2, x6, x5, x4, x3), # order 2 on vertices
    (x3, x4, x5, x6, x1, x2), # order 3 on vertices
    (x5, x6, x1, x2, x3, x4), # inverse of g3
    (-x1, x2, -x3, x4, -x5, x6)) # sign flip
for g in geometric_symmetries:
    assert(gram_determinant(*g) == gram_determinant)

Construct a Regge symmetry of the Gram equation and confirm that it does preserve the ideal $\det(M)$.

In [10]:
regge_sub = (x1, x2, x4*x5*x6/x3, x3*x5*x6/x4, x3*x4*x6/x5, x3*x4*x5/x6) # Regge involution

def regge(f):
    g = f(*regge_sub)
    for i in range(2, 6):
        g = decimate(g, i, 2)
    return g

assert(regge(gram_determinant) == gram_determinant)

Compute the effects of these symmetries on monomials. We will sometimes refer to the elements of `monomials` as $(y_i, y_i^{-1})$ for $i=1,\dots,6$ (but remembering that Python indexes from 0).

In [11]:
g_effects = []
for i in range(6):
    if i == 0:
        l = [regge(x[0]) for x in monomials]
    else:
        g = geometric_symmetries[i-1]
        l = [x[0](*g) for x in monomials]
    l2 = []
    for j in range(6):
        for k in range(6):
            for t in range(2):
                if l[j] == monomials[k][t]:
                    l2.append((k,t))
    g_effects.append(tuple(l2))
assert(len(x) == 6 for x in g_effects)
print(g_effects)

[((0, 0), (2, 0), (1, 0), (3, 1), (4, 1), (5, 0)), ((5, 1), (1, 0), (2, 0), (3, 0), (4, 0), (0, 1)), ((0, 0), (2, 0), (1, 0), (4, 1), (3, 1), (5, 0)), ((1, 0), (2, 0), (0, 0), (5, 0), (3, 0), (4, 0)), ((2, 0), (0, 0), (1, 0), (4, 0), (5, 0), (3, 0)), ((0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0))]


Create a new polynomial ring with variables corresponding to the first entries of `monomials`, then construct substitutions corresponding to the symmetries listed above.

In [12]:
Q.<y1,y2,y3,y4,y5,y6> = LaurentPolynomialRing(QQ, 6, order='lex')
g_effects_on_y = tuple(tuple(Q.gens()[g_effects[i][j][0]]^(1 if g_effects[i][j][1]==0 else -1) for j in range(6))
                  for i in range(5))
print(g_effects_on_y)

((y1, y3, y2, y4^-1, y5^-1, y6), (y6^-1, y2, y3, y4, y5, y1^-1), (y1, y3, y2, y5^-1, y4^-1, y6), (y2, y3, y1, y6, y4, y5), (y3, y1, y2, y5, y6, y4))


Express the same as a matrix.

In [13]:
g_effects_as_matrix = []
for i in range(5):
    g = g_effects[i]
    g_effects_as_matrix.append(Matrix(6,6,{(g[j][0],j): (1 if g[j][1] == 0 else -1) for j in range(6)}))

Verify the group generated by these matrices is the full group of signed $6 \times 6$ permutation matrices with an even number of minus signs, i.e., the Weyl group $W(D_6)$. (This is not used in what follows.)

In [14]:
assert(MatrixGroup(g_effects_as_matrix).order() == 2^5 * factorial(6))

Construct our list of target base fields for solutions.

In [15]:
target_fields = set([42, 48, 120])

## Function declarations: iteration over cosine relations

Utility function for computing $(a-b)^p / (a-b)$.

In [16]:
def cyclo_sum(a, b, p):
    return sum(a^i*b^(p-1-i) for i in range(p))

Given a string `iter_type` representing a type of mod-2 cosine relation and a list `part` of indices into `monomials`, yield tuples of relations corresponding to the various ways that a cosine relation of this type could be imposed on these monomials. 

This requires accounting for permuting the monomials as for individual inversions/negations.

The string `iter_type` consists of the length of the relation plus possibly one of the following symbols:
- "*": indicates a relation with a free parameter. (This corresponds to the symbol $'$ in the paper.)
- "a" or "b": disambiguates between two relations of the same length (neither with a free parameter).

We only implement relation types that can occur in a 6-term relation with at least one free parameter. This excludes types of length 5 and 6 with no free parameter.

Python note: the keyword `yield` is similar to `return`, but has the effect of making the function a generator rather than returning a list.

In [17]:
def relation_list(iter_type, part):
    for t in product([-1,1], repeat=len(part)-1):
        for sg in product([-1,1], repeat=len(part)-1):
            mons = (Q.gens()[part[0]-1],) + tuple(Q.gens()[part[i+1]-1]^t[i]*sg[i] for i in range(len(part)-1))
            if iter_type == "1a":
                yield (mons[0]^2 + 1,)
            elif iter_type == "1b":
                yield (mons[0]^2 - 1),
            elif iter_type == "2*":
                yield (mons[0] - mons[1],)
            elif iter_type == "3":
                for j in range(3): # -z3, z5, z5^2
                    (i1, i2) = (i for i in range(3) if i != j)
                    yield (cyclo_sum(mons[j], -1, 3), cyclo_sum(mons[i1], mons[i2], 5)) + \
                          tuple(cyclo_sum(mons[i], 1, 5) for i in (i1,i2))
            elif iter_type == "3*": # *, * z3, * z3^2
                yield (sum(mons[i] for i in range(3)),) + \
                      tuple(cyclo_sum(mons[i], mons[j], 3) for (i,j) in combinations(range(3), 2))
            elif iter_type == "4a":
                for c in permutations(range(4), int(2)): # -z3, -z3*z5, -z3^2*z5, z5^2
                     yield (sum(mons[i] + ~mons[i] for i in range(4)),
                            cyclo_sum(mons[c[0]], -1, 3), cyclo_sum(mons[c[1]], 1, 5)) + \
                         tuple(cyclo_sum(mons[i]^2, mons[c[1]], 3) for i in range(4) if i not in c)
            elif iter_type == "4b":
                for j in range(4): # -z3, z7, z7^2, z7^3
                    c = tuple(i for i in range(4) if i != j)
                    yield (cyclo_sum(mons[j], -1, 3), 1 + sum(mons[i] + ~mons[i] for i in c)) + \
                        tuple(cyclo_sum(mons[i], 1, 7) for i in c) + \
                        tuple(cyclo_sum(mons[i1], mons[i2], 7) for (i1,i2) in combinations(c, 2))
            elif iter_type == "5*": # *, * z5, * z5^2, * z5^3, * z5^4
                yield (sum(mons[i] for i in range(5)),) + \
                    tuple(cyclo_sum(mons[i], mons[j], 5) for (i,j) in combinations(range(5), 2))
            elif iter_type == "6*": 
                for c in combinations(range(6), 2): # -z3 *, -z3^2 *, z5 *, z5^2 *, z5^3 *, z5^4 *
                    c1 = tuple(i for i in range(6) if i not in c)
                    yield (sum(mons[i] for i in range(6)), cyclo_sum(mons[c[0]], mons[c[1]], 3)) + \
                        tuple(cyclo_sum(mons[c1[i]], mons[c1[j]], 5) for (i,j) in combinations(range(4), 2))
            else: # Not implemented: forms 5, 6
                raise ValueError("Form not supported")

Given `form`, a list of strings representing types of mod-2 cosine relations (of total length 6), yield all mod-2 cosine relations on `monomials` of this form.

This involves partitioning $\{1,\dots,6\}$ according to the form and then concatenating the results of applying `relation_list` to each part.

In [18]:
def relations_by_form(form):
    shape = []
    for s in form:
        s1 = s[:-1] if s[-1] in ('*', 'a', 'b') else s
        shape.append(int(s1))
    set_parts = OrderedSetPartitions(6, shape)
    ans = []
    for sp in set_parts:
        for t in product(*[relation_list(form[i], list(sp[i])) for i in range(len(form))]):
            yield tuple(chain(*t))

Given `form`, yield ideals in the `y` variables corresponding to mod-2 cosine relations of this form, by translating the results of `relations_by_form`.

We include some redundant ideal generators in order to maximize the visibility of symmetries, to simplify the symmetry reduction step.

In [19]:
def ideals_by_form(form):
    M = -identity_matrix(6)
    for u in relations_by_form(form):
        l2 = []
        for x in u:
            l2.append(x)
            l2.append(-x)
            xx = x.toric_coordinate_change(M) # Invert every variable
            l2.append(xx)
            l2.append(-xx)
        yield frozenset(l2)

## Symmetry reduction

#### In this section, we define subroutines to compute orbits for the action of $W(D_6)$.

Given a collection of ideals in $\mathbb{Q}[y_1^{\pm},\dots,y_6^{\pm}]$, return orbit representatives for the action of $W(D_6)$ by signed permutations.

We compute orbits by constructing the Cayley graph for some generators of $W(D_6)$ (the entries of `geometric_symmetries` plus `regge`), then taking connected components. Crucially, we start with normalized sets of generators and apply only toric transformations, so we can compare ideals using generators; this avoids a costly Groebner basis computation.

To reduce redundancy, each time we compute an edge, we make a note not to compute the reverse edge.

In [20]:
def compute_symmetry_action_in_y(form):
    # The list gens contains all ideal-generating tuples encountered so far.
    gens = []
    # The dictionary d1 will be keyed by ideal-generating tuples.
    # The corresponding value records the position of this tuple in gens.
    d1 = {}
    # The dictionary d will be keyed by ideal generating tuples, corresponding to vertices of the Cayley graph.
    # The corresponding value is the list of which group generators have already been used at that vertex.
    d = {}
    # The list edges contains all edges of the Cayley graph computed so far.
    edges = []
    for Jgens in ideals_by_form(form):
        # Record this ideal if we have not seen it yet.
        if not Jgens in d:
            d[Jgens] = []
            d1[Jgens] = len(gens)
            gens.append(Jgens)
        # For each group generator, construct the corresponding edge from Jgens.
        for g in range(5):
            # Skip this edge if we already have it.
            if g in d[Jgens]:
                continue
            # Compute the other endpoint of the edge.
            M = g_effects_as_matrix[g]
            J1gens = frozenset(x.toric_coordinate_change(M) for x in Jgens)
            # Record this ideal if we have not seen it yet.
            if J1gens not in d:
                d[J1gens] = []
                d1[J1gens] = len(gens)
                gens.append(J1gens)
            # Record the new edge.
            edges.append((d1[Jgens], d1[J1gens]))
            # Update d so that we do not compute this edge again.
            # Note that all of our generators have order 2 or 3. If the latter,
            # we can skip the third edge of the triangle.
            d[Jgens].append(g)
            d[J1gens].append(g)
            if g in (3,4):
                d[J1gens].append(7-g)
    Gamma = Graph(edges, loops=True)
    return [gens[l[0]] for l in Gamma.connected_components()]


The same, but in the `x` variables. For this, we must normalize generators using Groebner bases.

One difference: here we only use geometric symmetries, not Regge symmetries.

If `return_orbits` is True, we return complete orbits rather than representatives.

In [21]:
def compute_symmetry_action_in_x(l, return_orbits=False):
    if not l:
        return []
    P = l[0].ring()
    K = P.base_ring()
    # The list vertices contains the vertices added to the Cayley graph so far.
    vertices = []
    # The dict d keys each generating tuple to its corresponding ideals.
    d = {}
    # The dict d2 keys each generating tuple to a list of group generators already used at that vertex.
    d2 = {}
    # Go through the ideals in l, normalizing generators and updating d and d2.
    for J in l:
        J2 = J.normalize_gens()
        J2gens = J2.gens()
        d[J2gens] = J2
        d2[J2gens] = []
    # The list new_vertices contains vertices to be added to the Cayley graph.
    new_vertices = list(d.keys())
    # The list edges contains all edges of the Cayley graph computed so far.
    edges = []
    # The tuple glist consists of the group operations, defined as functions on ring elements.
    glist = tuple(lambda x, s=s: x(*s) for s in geometric_symmetries)
    while new_vertices:
        Jgens = new_vertices.pop(0)
        vertices.append(Jgens)
        J = d[Jgens]
        # For each group generator, construct the corresponding edge from Jgens.
        for g in glist:
            if g in glist and g in d2[Jgens]:
                continue
            J1 = J.apply_map(g)
            # Normalize generators in the resulting ideal.
            J1 = J1.normalize_gens()
            J1gens = J1.gens()
            # Record this ideal if we have not seen it yet.
            if J1gens not in d:
                d[J1gens] = J1
                d2[J1gens] = []
                new_vertices.append(J1gens)
            # Record the new edge.
            edges.append((Jgens, J1gens))
            # Update d2 so that we do not compute this edge again.
            # Note that all of our generators have order 2 or 3. If the latter,
            # we can skip the third edge of the triangle.
            d2[Jgens].append(g)
            d2[J1gens].append(g)
            if g == glist[3]:
                d2[J1gens].append(glist[4])
            elif g == glist[4]:
                d2[J1gens].append(glist[3])
    Gamma = Graph([vertices, edges], loops=True)
    if return_orbits:
        return [[d[i] for i in l] for l in Gamma.connected_components()]
    return [d[l[0]] for l in Gamma.connected_components()]


## Solving the Gram equation from mod-2 relations

#### In this section, we define subroutines to compute solutions of the Gram equation compatible with a given mod-2 cosine relation.

Utility function to do the following.
- Given a possible form of a mod-2 cosine relation, reduce the list of subproblems (written in terms of the $y_i$) by symmetry, then substitute for the $x_i$.
- If `separate` is true, split into torsion cosets and eliminate degenerate cases.

In [22]:
def consolidate(form, separate=True):
    # Reduce by symmetries
    l1 = compute_symmetry_action_in_y(form)
    #print(len(l1)); sys.stdout.flush()
    # Substitute
    l2 = [P.ideal([x(*(monomials[i][0] for i in range(6))) for x in t]) for t in l1]
    if not separate:
        return l2
    # Separate into geometric components, then eliminate degenerate cases.
    l3 = []
    for I in l2:
        # print(I)
        l3 += torsion_closure(I, raw=True)
    #print(len(l3)); sys.stdout.flush()
    l4 = [I for I in l3 if not is_degenerate(I)]
    return l4

Given a collection of subproblems, yield a complete set of nondegenerate solutions up to Galois conjugation.

This involves calling out to `torsion_closure.sage`. The function `torsion_closure`, given an ideal, returns a list of prime ideals (defined over various cyclotomic number fields) which cover the torsion closure of the original ideal up to Galois conjugation.

To save time, we call `torsion_closure` in `raw` mode, which eliminates some cleanup steps to remove redundancy. One of these steps is to force each solution to be realized over its minimal field of definition; we thus manually impose this (using the function `contract_ideals_to_cyclotomic`) for those ideals confirmed as nondegenerate.

In [23]:
def solve_subproblems(l):
    for I in l:
        for J in torsion_closure(I+I.ring().ideal([gram_determinant]), raw=True):
            if not is_degenerate(J):
                yield contract_ideals_to_cyclotomic([J])[0][0]


Compute all solutions arising from a given form of a mod-2 cosine relation; certify that all sporadic solutions are defined over permitted cyclotomic fields; and return all positive-dimensional solutions up to equivalence and Galois conjugation.

For forms with two or more free parameters, we separate the ideals coming from mod-2 relations after changing from the `y` variables back to the `x` variables.
Otherwise, we skip this step as it is prohibitively expensive compared to computing extra torsion closures.

In [24]:
def certify_form(form):
    print("Form: {}".format(form))
    # Count free parameters in this form
    dim = sum(1 for s in form if "*" in s)
    # Count mod-2 relations of this form
    c = sum(1 for _ in relations_by_form(form))
    print("Number of mod-2 relations: {}".format(c)); sys.stdout.flush()
    l = consolidate(form, separate=(dim >= 2))
    if dim == 3:
        l, _ = extend_ideals_to_cyclotomic(l)
        l = compute_symmetry_action_in_x(l)
    s = ("", "unseparated", "separated", "separated, desymmetrized")[dim]
    print("Number of {} ideals from mod-2 relations: {}".format(s, len(l))); sys.stdout.flush()
    # Count solutions.
    count = 0
    # Track zeta orders of zero-dimensional solutions
    field_sizes = []
    # Record positive-dimensional solutions
    ans = []
    for I in solve_subproblems(l):
        count += 1
        if I.dimension() == 0:
            zo = I.base_ring().zeta_order()
            if zo not in field_sizes:
                field_sizes.append(zo)
        else:
            ans.append(I)
    print("Number of nondegenerate solutions: {}".format(count))
    print("Cyclotomic orders: {}".format(sorted(field_sizes)))
    # Check that all field sizes are compliant
    assert all(any(x % y == 0 for x in target_fields) for y in field_sizes)
    # Check that there are no solutions with free parameters
    print("Further analysis required" if ans else "Form certified")
    return ans

## Compilation of solutions

For each possible form of a six-term mod-2 cosine relation with at least one free parameter, use the function `certify_form` to compute all solutions of the Gram determinant equation up to $W(D_6)$-action and Galois conjugation.

Since these computations are lengthy, we put each form in a separate cell, so that the cells can be executed in any order or even in multiple sessions. In the latter case, first run all cells above this one in order to set up the necessary declarations.

In [25]:
assert not certify_form(["2*", "2*", "2*"])

Form: ['2*', '2*', '2*']
Number of mod-2 relations: 5760


Number of separated, desymmetrized ideals from mod-2 relations: 14


Number of nondegenerate solutions: 440
Cyclotomic orders: [12, 24, 48, 60]
Form certified


In [26]:
assert not certify_form(["3*", "3*"])

Form: ['3*', '3*']


Number of mod-2 relations: 5120


Number of separated ideals from mod-2 relations: 144


Number of nondegenerate solutions: 28
Cyclotomic orders: [120]
Form certified


In [27]:
assert not certify_form(["3*", "2*", "1a"])

Form: ['3*', '2*', '1a']


Number of mod-2 relations: 3840


Number of separated ideals from mod-2 relations: 64


Number of nondegenerate solutions: 0
Cyclotomic orders: []
Form certified


In [28]:
ans = certify_form(["3*", "2*", "1b"])

Form: ['3*', '2*', '1b']


Number of mod-2 relations: 3840


Number of separated ideals from mod-2 relations: 104


Number of nondegenerate solutions: 244
Cyclotomic orders: [12, 24, 48]
Further analysis required


In [29]:
assert not certify_form(["2*", "2*", "1a", "1b"])

Form: ['2*', '2*', '1a', '1b']


Number of mod-2 relations: 2880


Number of separated ideals from mod-2 relations: 64


Number of nondegenerate solutions: 32
Cyclotomic orders: [48]
Form certified


In [30]:
assert not certify_form(["6*"])

Form: ['6*']


Number of mod-2 relations: 15360


Number of unseparated ideals from mod-2 relations: 16


Number of nondegenerate solutions: 208
Cyclotomic orders: [60, 120]
Form certified


In [31]:
assert not certify_form(["5*", "1a"])

Form: ['5*', '1a']


Number of mod-2 relations: 1536


Number of unseparated ideals from mod-2 relations: 3


Number of nondegenerate solutions: 0
Cyclotomic orders: []
Form certified


In [32]:
assert not certify_form(["5*", "1b"])

Form: ['5*', '1b']


Number of mod-2 relations: 1536


Number of unseparated ideals from mod-2 relations: 3


Number of nondegenerate solutions: 0
Cyclotomic orders: []
Form certified


In [33]:
assert not certify_form(["4a", "2*"])

Form: ['4a', '2*']


Number of mod-2 relations: 46080


Number of unseparated ideals from mod-2 relations: 22


Number of nondegenerate solutions: 16
Cyclotomic orders: [30]
Form certified


In [34]:
assert not certify_form(["4b", "2*"])

Form: ['4b', '2*']


Number of mod-2 relations: 15360


Number of unseparated ideals from mod-2 relations: 14


Number of nondegenerate solutions: 64
Cyclotomic orders: [42]
Form certified


In [35]:
assert not certify_form(["3*", "3"])

Form: ['3*', '3']


Number of mod-2 relations: 15360


Number of unseparated ideals from mod-2 relations: 10


Number of nondegenerate solutions: 48
Cyclotomic orders: [120]
Form certified


In [36]:
assert not certify_form(["3", "2*", "1a"])

Form: ['3', '2*', '1a']


Number of mod-2 relations: 11520


Number of unseparated ideals from mod-2 relations: 10


Number of nondegenerate solutions: 0
Cyclotomic orders: []
Form certified


In [37]:
assert not certify_form(["3", "2*", "1b"])

Form: ['3', '2*', '1b']


Number of mod-2 relations: 11520


Number of unseparated ideals from mod-2 relations: 10


Number of nondegenerate solutions: 192
Cyclotomic orders: [30, 60, 120]
Form certified


The successful result certifies that all sporadic solutions arising from positive-dimensional solutions of the mod-2 equation have fields of definition of the desired form.

## Further analysis of one-dimensional solutions

The form $[3^*, 2^*, 1]$ led to some one-dimensional parametric solutions. We check that the returned solutions lie in a single $G'$-orbit. (We only need geometric symmetries here, but remember that in a previous step we did reduce by the $G'$-action.)

In [38]:
ans2 = compute_symmetry_action_in_x(ans)
assert(len(ans2) == 1)

In [39]:
print(ans2[0])

Ideal (x_12 + (zeta12^3 - zeta12)*x_23, x_34 + (zeta12^3 - zeta12)*x_23, x_13 + (zeta12^2 - 1)*x_23, x_24 + (-zeta12^2 + 1)*x_23, x_14 + x_23) of Multivariate Laurent Polynomial Ring in x_12, x_34, x_13, x_24, x_14, x_23 over Cyclotomic Field of order 12 and degree 4


We check that the chosen representative of this orbit is $G'$-equivalent to its Galois conjugates.

In [40]:
K = ans2[0].base_ring()
ans3 = compute_symmetry_action_in_x([ans2[0].apply_coeff_map(h) for h in K.automorphisms()])
assert(len(ans3) == 1)

The successful result certifies that all positive-dimensional parametric solutions form a single $G'$-orbit.